In [1]:

from dotenv import load_dotenv

load_dotenv()

import os
import json
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings
from pymongo import MongoClient
from langchain_core.documents import Document

In [2]:
client = MongoClient(os.environ["SPACE_VECTOR_STORE_MONGO_CONN_STR"])


In [3]:
query = "서교동"

In [6]:
vector_store = MongoDBAtlasVectorSearch(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536),
    relevance_score_fn="cosine",
    collection=client["vector_store"]["area"],
    index_name="area_name_vector_index",
    text_key="name",
    embedding_key="name_embedding",
)

# vector_store.create_vector_search_index(dimensions=1536)

cursor = client["daytrip_area"]["node"].find({}, {"_id": 1, "name": 1})
documents = []
for doc in cursor:
    documents.append(Document(
        page_content=json.dumps(doc["name"], ensure_ascii=False),
        metadata={"area_id": str(doc["_id"])}
    ))
    
vector_store.add_documents(documents=documents)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10048bfe0>>
Traceback (most recent call last):
  File "/Users/sookhyun/Documents/personal/langchain-rag-poc/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
KeyboardInterrupt: 

KeyboardInterrupt



In [42]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 100})

documents = retriever.invoke("서교동")
for doc in documents:
    print(doc.page_content)

{"ko": "상교동", "en": "Sangyo-dong", "id": "Sangyo-dong"}
{"ko": "교동", "en": "Gyo-dong", "id": "Gyo-dong"}
{"ko": "교동", "en": "Gyo-dong", "id": "Gyo-dong"}
{"en": "주교동", "zh": "舟橋洞"}
{"ko": "석교동", "en": "Seokgyo-dong", "id": "Seokgyo-dong"}
{"ko": "팔당대교", "en": "Paldang Bridge"}
{"ko": "교하동", "en": "Gyoha-dong", "id": "Gyoha-dong"}
{"ko": "삼선동", "en": "Samseon-dong"}
{"ko": "동대구역", "en": "Dongdaegu Station"}
{"ko": "관교동", "en": "Gwangyo-dong", "id": "Gwangyo-dong"}
{"ko": "용산역", "en": "Yongsan Station", "id": "Yongsan Station"}
{"ko": "서울역", "en": "Seoul station"}
{"en": "Songyo District", "ru": "округ Сонгё", "ko": "선교구역", "ja": "船橋区域"}
{"ko": "소공동", "en": "Sogong-dong", "id": "Sogong-dong"}
{"ko": "동선동", "en": "Dongseon-dong", "id": "Dongseon-dong"}
{"ko": "삼성역", "en": "Samseong Station"}
{"ko": "대사동", "en": "Daesa-dong", "id": "Daesa-dong"}
{"en": "Samseong-dong", "ko": "삼성동", "zh": "三聖洞"}
{"ko": "고산동", "en": "Gosan-dong", "id": "Gosan-dong"}
{"en": "Gyesu-dong", "ko": "계수동", "ja": "桂

In [68]:
cursor = client["daytrip_area"]["node"].aggregate([
  {
    "$search": {
      "index": "area_node_name_search",
      "autocomplete": {
        "query": query,
        "path": "name.en",
        "fuzzy": {
          "maxEdits": 2,
          # "maxExpansions": 20
        }
      }
    }
  },
    {"$limit": 20},
    {"$project": {"_id": 1, "name": 1}}
])

for doc in cursor:
    print(doc)

{'_id': ObjectId('6475c21cc14f1f5f92386ae1'), 'name': {'ko': '잠실 / 송리단길', 'en': 'Jamsil / Songnidan-gil', 'id': 'Jamsil / Songnidan-gil'}}
